In [1]:
import numpy as np
import scipy as sp
from sympy import *
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from matplotlib.gridspec import GridSpec 
from mpl_toolkits.mplot3d import Axes3D
from scipy.linalg import block_diag
from scipy.integrate import odeint
%config InlineBackend.figure_format = 'retina'
import pandas as pd

In [2]:
#define dynamical system
def logistic_map(x,r):
    return x*r*(1-x)

In [3]:
def logistic_map_bifurcation(x_0, n, n_skip):
    #r_list = [2.5, 2.75, 3, 3.25, 3.5, 3.75, 3.8, 3.85, 3.9, 3.95]
    r_list = [3.9]
    R = []
    X = []
    for r in r_list:
        x = x_0
        for i in range(n):
            if i>=n_skip:
                X.append(x)
                R.append(r)
            x = logistic_map(x, r)
    return X,R

In [4]:
X, R = logistic_map_bifurcation(0.5, 1000, 0)

In [5]:
dx = np.array(X[1:])
X = np.array(X)

In [6]:
# Non-linear function library
library = PolynomialFeatures(degree=5, include_bias=True)
# include_bias parameter is for the feature in which all polynomial powers are zero - column of ones
Theta = library.fit_transform(X.reshape(-1,1)[:-1])
function_library = pd.DataFrame(data=Theta, columns=library.get_feature_names())

In [7]:
function_library.head()

,1,x0,x0^2,x0^3,x0^4,x0^5
0,1.0,0.500000,0.250000,0.125000,0.062500,0.031250
1,1.0,0.975000,0.950625,0.926859,0.903688,0.881096
2,1.0,0.095063,0.009037,0.000859,0.000082,0.000008
3,1.0,0.335500,0.112560,0.037764,0.012670,0.004251
4,1.0,0.869465,0.755969,0.657289,0.571490,0.496890


In [8]:
coeff, residuals, rank, s = np.linalg.lstsq(Theta, dx, rcond=None)
treshold = 0.5
for k in range(5):
    coeff, residuals, rank, s = np.linalg.lstsq(Theta, dx, rcond=None) 
    coeff[np.abs(coeff) < treshold] = 0

In [9]:
coeff_df = pd.DataFrame(data=coeff, index=library.get_feature_names())
print('Total number of possible coefficients:', coeff.size)
print('Number of non-zero coefficients:', np.count_nonzero(coeff))

Total number of possible coefficients: 6
Number of non-zero coefficients: 2


In [10]:
coeff_df

,0
1,0.0
x0,3.9
x0^2,-3.9
x0^3,0.0
x0^4,0.0
x0^5,0.0
